<a href="https://colab.research.google.com/github/Guimol/TBL-Brill/blob/main/TBL_GUILHERME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Projeto de mestrado
### Algoritmo TBL-Brill

#Carregando o ambiente

In [2]:
!pip install conllu
!pip install pyconll

import conllu, pyconll, io

from conllu import parse_incr
from nltk import BrillTagger, BrillTaggerTrainer

#Neste diretório estão os recursos que utilizei para formar o bl_anotada_snl.conllu
##Está compartilhado com você e o Prof Evandro

In [3]:
from io import open
import requests

path = 'https://raw.githubusercontent.com/Guimol/TBL-Brill/main/Corpus/bl_anotada_snl.conllu'

r = requests.get(path, allow_redirects=True)
open('conll_', 'wb').write(r.content)

bl = pyconll.load_from_file('conll_')

In [ ]:
#Acredito que era algo assim que voce queria que fosse o algoritmo para encontro de snl e palavras
#O erro aparentemente eh uma bobeira feito pelo proprio python, onde quando voce quer armazenar uma lista e em seguida limpa-la aonde ela foi armazenada tambem eh limpado automaticamente.

snl = list()
lista_palavras = list()
lista_snl = list()
lista_frases = list()

for sentence in bl:
  for word in sentence:
    if word.xpos == 'B' or word.xpos == 'I':
      snl.append((word.lemma, word.deprel))
    
    lista_palavras.append((word.lemma, word.deprel))

  lista_snl.append(list(snl))
  lista_frases.append(list(lista_palavras))

  snl.clear()
  lista_palavras.clear()

In [ ]:
i, j = 0, 0

for frase in lista_snl:
  for tupla in frase:
    if tupla[0] == None or tupla[1] == None:
      lista_snl[i].pop(j)
    j += 1
  j = 0
  i += 1

In [30]:
print(lista_snl)

[[('o', 'det'), ('avaliação', 'root'), ('de', 'case'), ('o', 'det'), ('Instituto', 'nmod'), ('Internacional', 'flat:name'), ('de', 'case'), ('Estudos', 'nmod'), ('Estratégicos', 'flat:name'), ('de', 'case'), ('Londres', 'nmod')], [('o', 'det'), ('acordo', 'nsubj'), ('de', 'case'), ('o', 'det'), ('dívida', 'nmod'), ('o', 'det'), ('troca', 'obj'), ('de', 'case'), ('o', 'det'), ('papel', 'nmod'), ('atual', 'amod'), ('combinação', 'nmod'), ('de', 'case'), ('cinco', 'nummod'), ('tipo', 'nmod'), ('diferente', 'amod'), ('de', 'case'), ('papel', 'nmod')], [('o', 'det'), ('volante', 'obl'), ('o', 'det'), ('ataque', 'obj'), ('de', 'case'), ('o', 'det'), ('São', 'nmod'), ('Paulo', 'flat:name'), ('de', 'case'), ('o', 'det'), ('técnico', 'nmod')], [('o', 'det'), ('reunião', 'nsubj'), ('Budapeste', 'obl'), ('.', 'punct')], [('CONCERTOS', 'root'), ('INTERNACIONAIS', 'flat:name'), ('GLOBO', 'flat:name')], [('o', 'det'), ('presidente', 'obl'), ('de', 'case'), ('o', 'det'), ('Corecon', 'nmod'), ('o', 'd

#Brill Tagger

In [31]:
frases_treino = lista_frases[:int(0.7 * len(lista_frases))]
frases_teste = lista_frases[int(0.7 * len(lista_frases)):]

In [33]:
import nltk

In [34]:
t0 = nltk.DefaultTagger('DT')
t1 = nltk.UnigramTagger(frases_treino, backoff=t0)
t2 = nltk.BigramTagger(frases_treino, backoff=t1)
t3 = nltk.TrigramTagger(frases_treino, backoff=t2)

In [35]:
# Loading Libraries
from nltk.tag import brill, brill_trainer

def train_brill_tagger(initial_tagger, train_sents, **kwargs):
	templates = [
			brill.Template(brill.Pos([-1])),
			brill.Template(brill.Pos([1])),
			brill.Template(brill.Pos([-2])),
			brill.Template(brill.Pos([2])),
			brill.Template(brill.Pos([-2, -1])),
			brill.Template(brill.Pos([1, 2])),
			brill.Template(brill.Pos([-3, -2, -1])),
			brill.Template(brill.Pos([1, 2, 3])),
			brill.Template(brill.Pos([-1]), brill.Pos([1])),
			brill.Template(brill.Word([-1])),
			brill.Template(brill.Word([1])),
			brill.Template(brill.Word([-2])),
			brill.Template(brill.Word([2])),
			brill.Template(brill.Word([-2, -1])),
			brill.Template(brill.Word([1, 2])),
			brill.Template(brill.Word([-3, -2, -1])),
			brill.Template(brill.Word([1, 2, 3])),
			brill.Template(brill.Word([-1]), brill.Word([1])),
			]
	
	# Using BrillTaggerTrainer to train
	trainer = brill_trainer.BrillTaggerTrainer(
			initial_tagger, templates, deterministic = True)
	
	return trainer.train(train_sents, **kwargs)


In [36]:
brill_tag = train_brill_tagger(t3, frases_treino)
b = brill_tag.evaluate(frases_teste)

print ("Accuracy of brill_tag : ", b)

Accuracy of brill_tag :  0.5454545454545454


In [ ]:
#Links de ref
#https://www.nltk.org/book/ch05.html
#https://www.nltk.org/api/nltk.tag.sequential.html?highlight=unigramtagger#nltk.tag.sequential.UnigramTagger
#https://www.nltk.org/api/nltk.tag.brill.html?highlight=brill#module-nltk.tag.brill
#https://www.nltk.org/api/nltk.tag.brill_trainer.html?highlight=brill#module-nltk.tag.brill_trainer
#https://www.geeksforgeeks.org/nlp-brill-tagger/